# Pattern Mining: Discovering Hidden Symptom Relationships 🔬

**Goal**: Find co-occurring symptoms that clinical trials miss

**Learning Concepts**:
- Association Rule Mining (Apriori algorithm)
- Support, Confidence, and Lift metrics
- Pattern discovery in real-world data
- Temporal pattern extraction

**Example Discoveries**:
- "yeast_infection + vaginal_dryness + long_term_use" (Support: 45 posts, Confidence: 78%)
- "post_pill + acne + hair_loss" (Support: 32 posts, Confidence: 65%)

In [ ]:
# Setup
import sys
sys.path.append('../src')

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
from tqdm.notebook import tqdm  # Progress bars for Jupyter

# Our custom analyzers
from analysis.medical_term_extractor import MedicalTermExtractor
from analyzers.association_rules import AssociationRulesMiner

# Styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Libraries loaded!")

## Step 1: Load Data

We collected 537 posts from 4 subreddits with expanded symptom keywords.

In [ ]:
# Load the newest data file
data_path = Path('../data/raw/reddit_bc_symptoms_posts_20251027_175721.json')

with open(data_path, 'r', encoding='utf-8') as f:
    posts = json.load(f)

print(f"📊 Loaded {len(posts)} posts")
print(f"   Subreddits: {set(p['subreddit'] for p in posts)}")
print(f"   Total text: {sum(p['text_length'] for p in posts):,} characters")
print(f"   Average score: {sum(p['score'] for p in posts) / len(posts):.1f}")

In [ ]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(posts)
df['created_date'] = pd.to_datetime(df['created_utc'], unit='s')

# Combine title and selftext for full analysis
df['full_text'] = df['title'] + ' ' + df['selftext']

print("\n📈 Data Overview:")
print(df[['subreddit', 'score', 'num_comments', 'text_length']].describe())

## Step 2: Extract Symptoms

Use our expanded `MedicalTermExtractor` to identify:
- **Mental symptoms**: depression, anxiety, mood swings, brain fog, etc.
- **Physical symptoms**: acne, yeast infections, dryness, hair loss, etc.
- **Temporal markers**: long-term use, just started, stopped, post-pill
- **User context**: first-time user, switcher, long-term user

In [ ]:
# Initialize extractor
extractor = MedicalTermExtractor()

print("🔬 Extracting symptoms from all posts...")
print("   Watch the progress bar below!\n")

# Extract symptoms for each post
analyzed_posts = []

for post in tqdm(posts, desc="Extracting symptoms", unit="post"):
    full_text = post['title'] + ' ' + post['selftext']
    
    analysis = {
        'post_id': post['id'],
        'subreddit': post['subreddit'],
        'score': post['score'],
        'created_date': post['created_date'],
        
        # Extract all symptoms
        'symptoms': extractor.extract_symptoms(full_text, category='all'),
        'mental_symptoms': extractor.extract_symptoms(full_text, category='mental'),
        'physical_symptoms': extractor.extract_symptoms(full_text, category='physical'),
        
        # Extract temporal and user context
        'temporal_context': extractor.extract_temporal_context(full_text),
        'user_context': extractor.extract_user_context(full_text),
        
        # Extract birth control types
        'bc_types': extractor.extract_bc_types(full_text),
    }
    
    analyzed_posts.append(analysis)

print(f"\n✓ Extracted symptoms from {len(analyzed_posts)} posts!")

In [ ]:
# Calculate summary statistics
posts_with_symptoms = [p for p in analyzed_posts if len(p['symptoms']) > 0]
mental_only = [p for p in analyzed_posts if len(p['mental_symptoms']) > 0 and len(p['physical_symptoms']) == 0]
physical_only = [p for p in analyzed_posts if len(p['physical_symptoms']) > 0 and len(p['mental_symptoms']) == 0]
both = [p for p in analyzed_posts if len(p['mental_symptoms']) > 0 and len(p['physical_symptoms']) > 0]

print("\n📊 Symptom Statistics:")
print(f"   Posts with symptoms: {len(posts_with_symptoms)} ({len(posts_with_symptoms)/len(analyzed_posts)*100:.1f}%)")
print(f"   Mental symptoms only: {len(mental_only)} posts")
print(f"   Physical symptoms only: {len(physical_only)} posts")
print(f"   Both mental + physical: {len(both)} posts")
print(f"\n   Average symptoms per post: {np.mean([len(p['symptoms']) for p in posts_with_symptoms]):.1f}")

In [ ]:
# Most common symptoms overall
all_symptoms = Counter()
for post in analyzed_posts:
    for symptom in post['symptoms'].keys():
        all_symptoms[symptom] += 1

print("\n🔝 Top 20 Most Common Symptoms:")
for symptom, count in all_symptoms.most_common(20):
    pct = (count / len(posts_with_symptoms)) * 100
    print(f"   {symptom:25} {count:4} posts ({pct:5.1f}%)")

## Step 3: Association Rule Mining

**Goal**: Find symptom combinations that appear together more often than random chance.

**Metrics**:
- **Support**: How many posts have this pattern? (minimum: 3% = ~16 posts)
- **Confidence**: If symptom A appears, what % also have symptom B? (minimum: 60%)
- **Lift**: How much stronger is this pattern than random? (minimum: 1.2x)

In [ ]:
# Initialize pattern miner with thresholds
miner = AssociationRulesMiner(
    min_support=int(len(posts_with_symptoms) * 0.03),  # 3% of posts = ~16 posts
    min_confidence=0.60,  # 60% confidence
    min_lift=1.2  # 20% better than random
)

print(f"🔍 Mining patterns with:")
print(f"   Min support: {miner.min_support} posts ({miner.min_support/len(posts_with_symptoms)*100:.1f}%)")
print(f"   Min confidence: {miner.min_confidence:.0%}")
print(f"   Min lift: {miner.min_lift}x")
print("\n   This will take 1-2 minutes...\n")

# Run the Apriori algorithm!
rules = miner.find_patterns(analyzed_posts)

print(f"\n✓ Found {len(rules)} association rules!")

## Step 4: Analyze Discovered Patterns

Let's explore the strongest symptom relationships!

In [ ]:
# Convert rules to DataFrame for easier analysis
rules_df = pd.DataFrame(rules)

if len(rules_df) > 0:
    # Format for display
    rules_df['antecedent_str'] = rules_df['antecedent'].apply(lambda x: ' + '.join(x))
    rules_df['consequent_str'] = rules_df['consequent'].apply(lambda x: ' + '.join(x))
    rules_df['rule'] = rules_df['antecedent_str'] + ' → ' + rules_df['consequent_str']
    
    print("\n🏆 Top 20 Strongest Patterns (by Lift):\n")
    for idx, rule in rules_df.head(20).iterrows():
        print(f"{rule['antecedent_str']:40} → {rule['consequent_str']:30}")
        print(f"   Support: {rule['support']:3} posts ({rule['support_pct']:5.1f}%)")
        print(f"   Confidence: {rule['confidence']:5.1%}  |  Lift: {rule['lift']:.2f}x")
        print()
else:
    print("⚠️  No rules found matching the thresholds. Try lowering min_support or min_confidence.")

In [ ]:
# Visualize rule distribution
if len(rules_df) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Support distribution
    axes[0].hist(rules_df['support'], bins=20, edgecolor='black')
    axes[0].set_xlabel('Support (number of posts)')
    axes[0].set_ylabel('Number of Rules')
    axes[0].set_title('Support Distribution')
    axes[0].axvline(miner.min_support, color='red', linestyle='--', label=f'Min: {miner.min_support}')
    axes[0].legend()
    
    # Confidence distribution
    axes[1].hist(rules_df['confidence'], bins=20, edgecolor='black')
    axes[1].set_xlabel('Confidence')
    axes[1].set_ylabel('Number of Rules')
    axes[1].set_title('Confidence Distribution')
    axes[1].axvline(miner.min_confidence, color='red', linestyle='--', label=f'Min: {miner.min_confidence:.0%}')
    axes[1].legend()
    
    # Lift distribution
    axes[2].hist(rules_df['lift'], bins=20, edgecolor='black')
    axes[2].set_xlabel('Lift')
    axes[2].set_ylabel('Number of Rules')
    axes[2].set_title('Lift Distribution')
    axes[2].axvline(miner.min_lift, color='red', linestyle='--', label=f'Min: {miner.min_lift}x')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Rule Statistics:")
    print(f"   Average support: {rules_df['support'].mean():.1f} posts")
    print(f"   Average confidence: {rules_df['confidence'].mean():.1%}")
    print(f"   Average lift: {rules_df['lift'].mean():.2f}x")

## Step 5: Analyze Specific Patterns

Let's look at specific types of patterns:
- Mental + Physical co-occurrence
- Post-pill patterns
- Long-term use patterns
- Birth control type-specific patterns

In [ ]:
# Helper function to categorize symptoms
mental_symptoms = ['depression', 'anxiety', 'mood_swings', 'brain_fog', 'panic_attacks', 
                   'emotional_numbness', 'suicidal_thoughts', 'low_mood', 'irritability']

physical_symptoms = ['acne', 'yeast_infection', 'vaginal_dryness', 'hair_loss', 'weight_gain',
                     'bloating', 'low_libido', 'heavy_bleeding', 'spotting', 'cramping',
                     'headache', 'migraine', 'nausea']

def is_mental(symptom):
    return symptom in mental_symptoms

def is_physical(symptom):
    return symptom in physical_symptoms

# Find mental → physical rules
if len(rules_df) > 0:
    mental_physical_rules = rules_df[
        rules_df['antecedent'].apply(lambda x: any(is_mental(s) for s in x)) &
        rules_df['consequent'].apply(lambda x: any(is_physical(s) for s in x))
    ].sort_values('lift', ascending=False)
    
    print("\n🧠➡️💊 Mental → Physical Symptom Patterns:\n")
    if len(mental_physical_rules) > 0:
        for idx, rule in mental_physical_rules.head(10).iterrows():
            print(f"{rule['antecedent_str']:35} → {rule['consequent_str']:25}")
            print(f"   Support: {rule['support']} posts | Confidence: {rule['confidence']:.1%} | Lift: {rule['lift']:.2f}x\n")
    else:
        print("   No mental→physical patterns found")

In [ ]:
# Temporal pattern analysis
temporal_stats = Counter()
for post in analyzed_posts:
    for marker in post['temporal_context']:
        temporal_stats[marker] += 1

print("\n⏰ Temporal Patterns:")
for marker, count in temporal_stats.most_common():
    pct = (count / len(analyzed_posts)) * 100
    print(f"   {marker:20} {count:4} posts ({pct:5.1f}%)")

# User context analysis
context_stats = Counter()
for post in analyzed_posts:
    for ctx in post['user_context']:
        context_stats[ctx] += 1

print("\n👤 User Context:")
for ctx, count in context_stats.most_common():
    pct = (count / len(analyzed_posts)) * 100
    print(f"   {ctx:20} {count:4} posts ({pct:5.1f}%)")

# Birth control type analysis
bc_stats = Counter()
for post in analyzed_posts:
    for bc in post['bc_types']:
        bc_stats[bc] += 1

print("\n💊 Birth Control Types Mentioned:")
for bc, count in bc_stats.most_common():
    pct = (count / len(analyzed_posts)) * 100
    print(f"   {bc:25} {count:4} posts ({pct:5.1f}%)")

## Step 6: Export for Web Visualization

Export the discovered patterns in JSON format for the interactive web app.

In [ ]:
# Create output directory
output_dir = Path('../data/patterns')
output_dir.mkdir(exist_ok=True)

# Export network graph data for visualization
if len(rules) > 0:
    graph_output = output_dir / 'symptom_network.json'
    miner.export_for_visualization(rules, str(graph_output))
    print(f"✓ Exported network graph to {graph_output}")

# Export top patterns
patterns_output = output_dir / 'discovered_patterns.json'
if len(rules_df) > 0:
    top_patterns = rules_df.head(50)[[
        'antecedent', 'consequent', 'support', 'confidence', 'lift', 'support_pct'
    ]].to_dict('records')
    
    with open(patterns_output, 'w') as f:
        json.dump(top_patterns, f, indent=2)
    
    print(f"✓ Exported top 50 patterns to {patterns_output}")

# Export summary statistics
stats_output = output_dir / 'stats.json'
stats = {
    'total_posts': len(posts),
    'posts_with_symptoms': len(posts_with_symptoms),
    'total_rules_found': len(rules),
    'mining_params': {
        'min_support': miner.min_support,
        'min_confidence': miner.min_confidence,
        'min_lift': miner.min_lift
    },
    'top_symptoms': dict(all_symptoms.most_common(20)),
    'temporal_patterns': dict(temporal_stats),
    'user_contexts': dict(context_stats),
    'bc_types': dict(bc_stats),
    'symptom_categories': {
        'mental_only': len(mental_only),
        'physical_only': len(physical_only),
        'both': len(both)
    }
}

with open(stats_output, 'w') as f:
    json.dump(stats, f, indent=2)

print(f"✓ Exported statistics to {stats_output}")
print("\n✅ All pattern mining complete!")

## Summary & Next Steps

**What We Discovered**:
- Found hidden symptom co-occurrence patterns using association rule mining
- Identified mental + physical symptom relationships
- Extracted temporal patterns (long-term use, post-pill, etc.)
- Analyzed user contexts and birth control types

**Key Learnings**:
- **Support**: How common is a pattern?
- **Confidence**: If A appears, what % also have B?
- **Lift**: Is the pattern stronger than random chance?
- **Apriori Algorithm**: Efficiently builds patterns incrementally

**Next Steps**:
1. ✅ Pattern mining complete → Ready for visualization!
2. 🔜 Build temporal patterns analyzer (timeline extraction)
3. 🔜 Build network analysis module (community detection)
4. 🔜 Create mobile-friendly web app with interactive visualizations